# Running models and cross validation 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn import metrics

In [7]:
#Time series split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('')

In [ ]:
# Separating features and target variable
X = df(['Price'], axis=1)
y = df['Price']